## <center>CSE 546: Reinforcement Learning</center>
### <center>Prof. Alina Vereshchaka</center>
<!-- ### <center>Fall 2022</center> -->

Welcome to the Assignment 2, Part 1: Introduction to Deep Reinforcement Learning and Neural Networks! The goal of this assignment is to make you comfortable with the application of different Neural Network structures depending on how the Reinforcement Learning environment is set up.

In [3]:
# Imports
from environment import WumpusWorldEnvironment

# Environment

We will be working with an implementation of the Wumpus World environment. The environment comes from the book "Artificial Intelligence: A Modern Approach" by Stuart J. Russell and Peter Norvig. 

### ENVIRONMENT DETAILS:

The environment is a 6 x 6 grid world containing a total of 36 grid blocks. 

#### ENVIRONMENT OBJECTS:
The environment consists of the following objects:

1. **Agent** - The agent starts in the grid block at the bottom left corner whose co-ordinates are [0, 0]. The goal of our agent is to collect the Gold while avoiding the Wumpus and the pits. 

2. **Wumpus** - The monster which would eat the agent if they are in the same grid block.

3. **Pit** - The agent must avoid falling into the pits. 

4. **Gold** - The agent must collect the Gold.

5. **Breeze** - Breeze surrounds the Pits and warn the agent of a Pit in an adjacent grid block.

6. **Stench** - Stench surrounds the Wumpus and warns the agent of the Wumpus in an adjacent grid block.

#### ENVIRONMENT OBSERVATIONS:

Our implementation of the environment provides you with four different types of observations:

1. **Integer** - Integer in the range [0 - 35]. This represents the grid block the agent is in. E.g., if the agent is in the bottom left grid block (starting position) the observation would be 0, if the agent is in the grid block containing the Gold the observation would be 34, if the agent is in the top right grid block the observation would be 35.

2. **Vector** - 

    **2.1.** A vector of length 2 representing the agent co-ordinates. The first entry represents the x co-ordinate and the second entry represets the y co-ordinate. E.g., if the agent is in the bottom left grid block (starting position) the observation would be [0, 0], if the agent is in the grid block containing the Gold the observation would be [4, 5], if the agent is in the top right grid block the observation would be [5, 5].
    
    **2.2.** A vector of length 36 representing the one-hot encoding of the integer observation (refer type 1 above). E.g., if the agent is in the bottom left grid block (starting position) the observation would be [1, 0, ..., 0, 0], if the agent is in the grid block containing the Gold the observation would be [0, 0, ..., 1, 0], if the agent is in the top right grid block the observation would be [0, 0, ..., 0, 1].


3. **Image** - Image render of the environment returned as an NumPy array. The image size is 84 * 84 (same size used in the DQN paper). E.g., if the agent is in the bottom right grid block the observation is:

    Observation: (84 * 84)

     [[255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     ...

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]]

    Observation type: <class 'numpy.ndarray'>

    Observation Shape: (84, 84)

    Visually, it looks like:
    <img src="./images/environment_render.png" width="500" height="500">
    

4. **Float** - Float in the range [0 - $\infty$] representing the time elapsed in seconds. 

#### ENVIRONMENT ACTIONS:

Our implementation of the environment provides you with three different types of actions:

1. **Discrete** - Integer in the range [0 - 3] representing the four actions possible in the environment as follows: 0 - Right 1 - Left 2 - Up 3 - Down.

2. **Multi-Discrete** - Array of length four where each element takes binary values 0 or 1. Array elements represent if we take a particular action. Array element with index 0 corresponds to the right action, index 1 corresponds to the left action, index 2 corresponds to the up action, and index 3 corresponds to the down action. E.g., 
   action = [1, 0, 0, 0] would result in the agent moving right.
   action = [1, 0, 1, 0] would result in the agent moving right and up.
   action = [0, 1, 0, 1] would result in the agent moving left and down.

3. **Continuous** - Float in the range [-1, 1] determining whether the agent will go left, right, up, or down as follows:

    if -1 <= action <= -0.5:
        Go Right.
    elif -0.5 < action <= 0:
        Go Left.
    elif 0 < action <= 0.5:
        Go Up.
    elif 0.5 < action <= 1:
        Go Down.
        
### YOUR TASK IS TO USE A NEURAL NETWORK TO WORK WITH ALL FOUR TYPES OF OBSERVATIONS AND ALL THREE TYPES OF  ACTIONS.
### Note: You don't have to train your agent/neural network. You just have to build the neural network structure that takes the observation as input and produces the desired output with the initial weights.

#### You can use libraries such as PyTorch/TensorFlow/Keras to build your neural networks.

#### <span style="color:red">You cannot use RL libraries that already provide the neural network to you such as Stable-baselines3, Keras-RL, TF agents, Ray RLLib etc.</span>

<img src="./images/wumpus_world_environment.jpg" width="600" height="600">

# START COMPLETING YOUR ASSIGNMENT HERE

## Observation Type - Integer, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is an integer (refer to environment observations type 1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [4]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

model = nn.Sequential(
    nn.Linear(1, 64), # input=1, hidden=64
    nn.ReLU(),
    nn.Linear(64, 4) # hidden=64, output=4
)

# Normalization input  to [0, 1]
# Convert the integer observation (range [0, 35]) into a normalized float tensor
x = torch.tensor([[observation / 35.0]], dtype=torch.float32)

# Disable gradient computation since we are not training the model here (just a forward pass for prediction)
with torch.no_grad():
    # Pass the normalized observation through the neural network model to get Q-values
    q = model(x)                 # shape: [1, 4]
    q_np = q.numpy().squeeze()   # shape: [4]

# Pick the action of maximum Q value 
action = int(np.argmax(q_np))
action_map = {0: "Right", 1: "Left", 2: "Up", 3: "Down"}

print(f"obs = {observation}  -> normalized x = {x.item():.3f}")
print("Q-values [Right, Left, Up, Down] =", q_np)
print(f"Chosen action: {action} ({action_map[action]})")

# END_YOUR_CODE

obs = 0  -> normalized x = 0.000
Q-values [Right, Left, Up, Down] = [ 0.19290227 -0.3828006  -0.2255198   0.1122991 ]
Chosen action: 0 (Right)


## Observation Type - Vector (2.1), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 2 (refer to environment observations type 2.1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_2_64_4.png">

In [13]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='discrete')
observation, info = environment.reset()   # e.g., obs = [x, y] with x,y in {0..3}

# BEGIN_YOUR_CODE

# NN: in=2 → 64 → 4
model = nn.Sequential(
    nn.Linear(2, 64),
    nn.ReLU(),
    nn.Linear(64, 4)
)

# x,y normalize to [0,1] by dividing by 5; keep batch dim [1,2]
x = torch.tensor([observation], dtype=torch.float32) / 5.0  # shape [1,2]

# Forward pass (no gradient)
with torch.no_grad():
    q_values = model(x).numpy().squeeze()  # shape [4]

# Choose best action
action = int(np.argmax(q_values))
actions = {0: "Right", 1: "Left", 2: "Up", 3: "Down"}

# Print results
vals = x.squeeze(0).tolist()             # -> [x_norm, y_norm]
print(f"Obs: {observation} | Norm: ({vals[0]:.2f}, {vals[1]:.2f})")
print("Q-values:", q_values)
print(f"Best Action: {action} ({actions[action]})")

# END_YOUR_CODE

Obs: [0 0] | Norm: (0.00, 0.00)
Q-values: [0.15126288 0.22445862 0.53398895 0.09014611]
Best Action: 2 (Up)


## Observation Type - Vector (2.2), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4.png">

In [15]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

# int observation (0..35) -> one-hot [1,36]
x = F.one_hot(torch.tensor([observation]), num_classes=36).to(torch.float32)  # no normalization needed

# MLP: 36 -> 64 (ReLU) -> 4 (linear Q-values)
model = nn.Sequential(
    nn.Linear(36, 64),
    nn.ReLU(),
    nn.Linear(64, 4)
)

# Forward pass (inference only)
with torch.no_grad():
    q_values = model(x).numpy().squeeze()   # shape [4]

# Pick best action and print
actions = {0: "Right", 1: "Left", 2: "Up", 3: "Down"}
best = int(np.argmax(q_values))

print("Obs (int):", observation)
print("One-hot shape:", tuple(x.shape), "sum:", int(x.sum().item()))  # sanity check (should be 1)
print("Q-values:", q_values)
print(f"Best Action: {best} ({actions[best]})")


# END_YOUR_CODE

Obs (int): 0
One-hot shape: (1, 36) sum: 1
Q-values: [ 0.16534321 -0.13215527  0.14415725 -0.12879342]
Best Action: 0 (Right)


## Observation Type - Image, Action Type - Discrete

The part of the assignment requires you to create a convolutional neural network with one convolutional layer having 128 filters of size 3 x 3, one hidden layer having 64 neurons, and the output layer having 4 neurons. The input to the neural network is an image of size 84 * 84 (refer to environment observations type 3). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/convolutional_neural_network_84x84_128_64_4.png">

In [23]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='image', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

# observation: np.array (84×84). Convert to tensor [Batch size, Channels, Height, Width] and normalize to [0,1].
x = torch.tensor(observation, dtype=torch.float32) / 255.0   # Normalization / shape: [84,84], pixels: 0-255
x = x.unsqueeze(0).unsqueeze(0)                               # -> [1,1,84,84] (batch, channel)

# CNN: Conv(1->128, 3x3, padding=1 keeps 84x84) -> Flatten -> FC 64 -> FC 4 (linear Q-values)
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Flatten(),                               # [1, 128*84*84]
    nn.Linear(128*84*84, 64),
    nn.ReLU(),
    nn.Linear(64, 4)
)

# Forward pass (no gradients)
with torch.no_grad():
    q_values = model(x).numpy().squeeze()       # shape [4]

# Choose best action and print
actions = {0: "Right", 1: "Left", 2: "Up", 3: "Down"}
best = int(np.argmax(q_values))

print("Observation shape:", observation.shape)
print("Observation:\n", observation[:, :])   # observation 출력

print("Image tensor shape:", tuple(x.shape))    # (1,1,84,84)
print("Q-values:", q_values)
print(f"Best Action: {best} ({actions[best]})")

# import matplotlib.pyplot as plt
# plt.imshow(observation, cmap='gray'); plt.axis('off'); plt.show()

# END_YOUR_CODE

Observation shape: (84, 84)
Observation:
 [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
Image tensor shape: (1, 1, 84, 84)
Q-values: [-0.14458598 -0.12303841  0.02125098 -0.2295996 ]
Best Action: 2 (Up)


## Observation Type - Float, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a float (refer to environment observations type 4). The output of the neural network is an array representing the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [26]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='float', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

# MLP: 1 → 64 → 4
model = nn.Sequential(
    nn.Linear(1, 64),
    nn.ReLU(),
    nn.Linear(64, 4)
)

# Make input tensor; keep batch dim [1,1]
x = torch.tensor([[float(observation)]], dtype=torch.float32)
# (Optional) normalize if you know the range, e.g., x = (x - mean) / std

# Forward pass (no training)
with torch.no_grad():
    q_values = model(x).numpy().squeeze()   # shape [4]

# Pick best action & print
actions = {0: "Right", 1: "Left", 2: "Up", 3: "Down"}
best = int(np.argmax(q_values))

print("Observation (float):", observation)
print("Input tensor shape:", tuple(x.shape))
print("Q-values:", q_values)
print(f"Best Action: {best} ({actions[best]})")

# END_YOUR_CODE

Observation (float): 7.295608520507812e-05
Input tensor shape: (1, 1)
Q-values: [-0.24215408  0.03978841  0.11084732  0.40179184]
Best Action: 3 (Down)


## Observation Type - Vector (2.2), Action Type - Multi-Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array representing the probability of choosing the actions. (If the value of the array element is >=0.5 you will perform the action.) (refer to environment actions type 2).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4_sigmoid.png">

In [28]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action probabilities for the four actions. Print the observation and the action probabilities."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='multi_discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

# int -> one-hot [1,36]
x = F.one_hot(torch.tensor([observation]), num_classes=36).to(torch.float32)

# MLP: 36 -> 64 (ReLU) -> 4 (Sigmoid -> probabilities)
model = nn.Sequential(
    nn.Linear(36, 64),
    nn.ReLU(),
    nn.Linear(64, 4),
    nn.Sigmoid()                               # output in [0,1]
)

with torch.no_grad():
    probs = model(x).numpy().squeeze()         # shape [4], probabilities per direction

# Threshold 0.5 -> multi-discrete action vector
action_vec = (probs >= 0.5).astype(int)        # e.g., [1,0,1,0]

dirs = ["Right", "Left", "Up", "Down"]
print("Obs (int):", observation)
print("One-hot sum:", int(x.sum().item()))     # sanity check: should be 1
print("Action probabilities [R,L,U,D]:", probs)
print("Chosen multi-discrete action:", action_vec, 
      "=>", [d for d,flag in zip(dirs, action_vec) if flag==1])

# END_YOUR_CODE

Obs (int): 0
One-hot sum: 1
Action probabilities [R,L,U,D]: [0.49691388 0.5012522  0.49370664 0.48166177]
Chosen multi-discrete action: [0 1 0 0] => ['Left']


## Observation Type - Vector (2.2), Action Type - Continuous

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 1 neuron. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an float in the range [-1, 1] determining the action which will be taken. (refer to environment actions type 3).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector and use the TanH activation function to get the output in the range [-1, 1].

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_1.png">

In [5]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action. Print the observation and the predicted action."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='continuous')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

# Convert integer obs → one-hot vector [1,36]
x = F.one_hot(torch.tensor([observation]), num_classes=36).to(torch.float32)

# MLP: 36 → 64 → 1, with tanh output
model = nn.Sequential(
    nn.Linear(36, 64),
    nn.ReLU(),
    nn.Linear(64, 1),
    nn.Tanh()                       # output in [-1, 1]
)

# Forward pass (no gradients)
with torch.no_grad():
    action = model(x).item()        # scalar output

# Print observation & predicted action
print("Obs (int):", observation)
print("One-hot sum:", int(x.sum().item()))
print(f"Predicted continuous action: {action:.4f} (range [-1, 1])")

# END_YOUR_CODE

Obs (int): 0
One-hot sum: 1
Predicted continuous action: -0.0182 (range [-1, 1])


/opt/anaconda3/envs/env_torch/lib/python3.12/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
x

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])